In [1]:
from elasticsearch import Elasticsearch
from datetime import datetime as dt
import pandas as pd
import numpy as np
from csv import DictReader
import os

### Creamos la conexion con elastic

In [4]:
ELASTIC_PASSWORD ='vN0W3+GJfq8X4X9WgYFx' #haritz

client=Elasticsearch(
    "https://localhost:9200",
    ca_certs="/home/bdata2/elasticsearch-8.4.3/config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

client.info()

ObjectApiResponse({'name': 'bdata2-VirtualBox', 'cluster_name': 'elasticsearch', 'cluster_uuid': '6XmqEv5uSK-hr779sbmGdg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Dataframes de train y test

In [5]:
with open('./Datos/Transformados/df_train.csv', 'r') as fichero:
    csv_dict = DictReader(fichero)
    indice = 0
    for row in csv_dict:
        row['product_count'] = int(row['product_count'])
        row['recurrence'] = int(row['recurrence'])
        row['age_nan'] = int(row['age_nan'])
        row['duracion_reserva'] = int(row['duracion_reserva'])
        row['+80'] = int(row['+80'])
        row['16-18'] = int(row['16-18'])
        row['18-24'] = int(row['18-24'])
        row['25-35'] = int(row['25-35'])
        row['35-50'] = int(row['35-50'])
        row['50-65'] = int(row['50-65'])
        row['65-80'] = int(row['65-80'])
        row['All Iron Staff'] = int(row['All Iron Staff'])
        row['Business Group'] = int(row['Business Group'])
        row['Leisure Group'] = int(row['Leisure Group'])
        row['Leisure Individual'] = int(row['Leisure Individual'])
        row['Mid-Long Stay'] = int(row['Mid-Long Stay'])
        row['Travel Group'] = int(row['Travel Group'])
        row['Travel Individual'] = int(row['Travel Individual'])
        row['flexible'] = int(row['flexible'])
        row['non_refundable'] = int(row['non_refundable'])
        row['other'] = int(row['other'])
        row['lluvia'] = int(row['lluvia'])
        row['tormenta'] = int(row['tormenta'])
        row['libere'] = int(row['libere'])
        row['naitly'] = int(row['naitly'])
        row['aparthotel'] = int(row['aparthotel'])
        row['apartment'] = int(row['apartment'])
        row['cancellation'] = bool(row['cancellation'])
        
        resp = client.index(index = 'reto06_morado_df_train', id = indice, document=row)
        indice = indice + 1

In [6]:
resp=client.search(index="reto06_morado_df_train", query={"match_all":{}})
print(resp['hits']['total']['value'])

7401


In [7]:
with open('./Datos/Transformados/df_test.csv', 'r') as fichero:
    csv_dict = DictReader(fichero)
    indice = 0
    for row in csv_dict:
        row['product_count'] = int(row['product_count'])
        row['recurrence'] = int(row['recurrence'])
        row['age_nan'] = int(row['age_nan'])
        row['duracion_reserva'] = int(row['duracion_reserva'])
        row['+80'] = int(row['+80'])
        row['16-18'] = int(row['16-18'])
        row['18-24'] = int(row['18-24'])
        row['25-35'] = int(row['25-35'])
        row['35-50'] = int(row['35-50'])
        row['50-65'] = int(row['50-65'])
        row['65-80'] = int(row['65-80'])
        row['All Iron Staff'] = int(row['All Iron Staff'])
        row['Business Group'] = int(row['Business Group'])
        row['Leisure Group'] = int(row['Leisure Group'])
        row['Leisure Individual'] = int(row['Leisure Individual'])
        row['Mid-Long Stay'] = int(row['Mid-Long Stay'])
        row['Travel Group'] = int(row['Travel Group'])
        row['Travel Individual'] = int(row['Travel Individual'])
        row['flexible'] = int(row['flexible'])
        row['non_refundable'] = int(row['non_refundable'])
        row['other'] = int(row['other'])
        row['lluvia'] = int(row['lluvia'])
        row['tormenta'] = int(row['tormenta'])
        row['libere'] = int(row['libere'])
        row['naitly'] = int(row['naitly'])
        row['aparthotel'] = int(row['aparthotel'])
        row['apartment'] = int(row['apartment'])
        row['cancellation'] = bool(row['cancellation'])
        
        resp = client.index(index = 'reto06_morado_df_test', id = indice, document=row)
        indice = indice + 1

In [8]:
resp=client.search(index="reto06_morado_df_test", query={"match_all":{}})
print(resp['hits']['total']['value'])

3173


### Resultados del modelado sobre cancelaciones

In [9]:
with open('./Datos/Transformados/resultados.csv', 'r') as fichero:
    csv_dict = DictReader(fichero)
    indice = 0
    for row in csv_dict:
        row['highest_score'] = float(row['highest_score'])
        row['segundos'] = float(row['segundos'])
        row['hora_subida'] = pd.to_datetime(row['hora_subida'])
        resp = client.index(index = 'reto06_morado_resultados_modelos', id = indice, document=row)
        indice = indice + 1

In [10]:
resp=client.search(index="reto06_morado_resultados_modelos", query={"match_all":{}})
print(resp['hits']['total']['value'])


40


In [12]:
resp=client.search(index="reto06_morado_resultados_modelos", query={"range":{"highest_score":{"gte":0.94}}})
print(resp['hits']['total']['value'])
for i in range(0,resp['hits']['total']['value']):
    print(resp['hits']['hits'][i]['_source'])

6
{'model_used': 'LogisticRegression(max_iter=10000)', 'highest_score': 0.9432512728799022, 'best hyperparameters': "{'C': 1}", 'balanceo': 'sin_balanceo', 'segundos': 5.375005006790161, 'hora_subida': '2023-01-22T11:06:55.171158'}
{'model_used': 'SVC()', 'highest_score': 0.9416297425041517, 'best hyperparameters': "{'C': 2, 'kernel': 'rbf'}", 'balanceo': 'sin_balanceo', 'segundos': 24.082744359970093, 'hora_subida': '2023-01-22T11:07:13.878895'}
{'model_used': 'LogisticRegression(max_iter=10000)', 'highest_score': 0.9737500000000001, 'best hyperparameters': "{'C': 20}", 'balanceo': 'SMOTEENN(random_state=0)', 'segundos': 36.93844127655029, 'hora_subida': '2023-01-22T11:07:50.864655'}
{'model_used': 'SVC()', 'highest_score': 0.9779545454545454, 'best hyperparameters': "{'C': 3, 'kernel': 'rbf'}", 'balanceo': 'SMOTEENN(random_state=0)', 'segundos': 49.38101768493652, 'hora_subida': '2023-01-22T11:08:03.307213'}
{'model_used': 'LogisticRegression(max_iter=10000)', 'highest_score': 0.9429